![Curso Schwarz-Sosa-Suriano](http://www.fi.uba.ar/sites/default/files/logo.png)
# Integración Numérica - Tercera parte
**Curso Schwarz - Sosa - Suriano**
- Métodos Numéricos. *Curso 2*
- Análisis Numérico I. *Curso 4*
- Métodos Matemáticos y Numéricos. *Curso 6*

## Cuadratura de Gauss-Legendre

Este método propone una perspectiva totalmente distinta a la de los métodos de Newton-Cotes. En lugar de simplificar la elección de los $x_i$ tomando puntos equidistantes para obtener un sistema de ecuaciones lineales y obtener fácilmente los los $c_i$, *mantiene la totalidad de las incógnitas como tales con la finalidad de obtener una mejora en el orden del error*.


Para generar el Sistema de Ecuaciones Lineales de $2.n+2$ incógnitas y $2.n+2$ incógnitas, propone primero cambiar la variable $x \in [a,b]$ por  $t \in [-1,1]$:

$$ \int_a^b f(x) = \frac{b-a}{2} \int_{-1}^1 f\left(\frac{b-a}{2}t 
+ \frac{a+b}{2}\right)\,dt$$

Así las cosas, nuestra fórmula de cuadratura queda como:

$$ Q_n = \frac{b-a}{2} \sum_{i=0}^n c_i . f\left(\frac{b-a}{2}t_i + \frac{a+b}{2}\right)$$

Esta vez, al buscar las $2.n+2$ ecuaciones se aseguran resultados exactos al integrar polinomios de hasta grado $2.n+1$ *-lo cual implica además una mejora en el término de error de truncamiento.*

De la resolución del Sistema de Ecuaciones No Lineales para este nuevo intervalo normalizado surge que los $t_i$ coinciden con las raíces de los polinomios de Legendre, y los valores de los coeficientes $c_i$ se encuentran también relacionados con los mismos.

Aunque es posible calcular ambos valores en forma analítica, para aplicar el método se suele recurrir a los valores tabulados precalculados. De hecho, los tenemos disponibles en Numpy:

In [455]:
import numpy as np
import pandas as pd
tTabla, cTabla, pTabla, nTabla = [], [], [], []
for i in range (1,8):
    x, c = np.polynomial.legendre.leggauss(i)
    ok = False
    for k in range (len(x)):
        tTabla.append(x[k])
        cTabla.append(c[k])
        if (not ok):
            nTabla.append(i-1)
            pTabla.append(str(i)+' Puntos')
            ok = True
        else:
            pTabla.append('')
            nTabla.append('')

df = pd.DataFrame(data = {'$n$': nTabla, '$t_i$': tTabla, '$c_i$': cTabla}, index= pTabla)  

In [457]:
df

,$n$,$t_i$,$c_i$
1 Puntos,0,0.000000,2.000000
2 Puntos,1,-0.577350,1.000000
,,0.577350,1.000000
3 Puntos,2,-0.774597,0.555556
,,0.000000,0.888889
,,0.774597,0.555556
4 Puntos,3,-0.861136,0.347855
,,-0.339981,0.652145
,,0.339981,0.652145
,,0.861136,0.347855


In [458]:
import sympy as sm
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon
from ipywidgets import interactive,interact, HBox, Layout,VBox
import ipywidgets as widgets
from IPython.display import display, clear_output


def f(x):
    return x*np.sin(x)+x

a, b = 1,10

x = sm.symbols('x')
primitiva = sm.integrate(x * sm.sin(x) + x,x)
primitivaN = sm.lambdify(x,primitiva)
integralExacta = primitivaN(b)-primitivaN(a)
print ('Integral exacta ',integralExacta)

def ErrorRel(aprox):
    e=100*abs((aprox-integralExacta)/aprox)
    return e

Integral exacta  57.0455255009354


In [447]:
def graficoActualizar(a,b,f,q,k,n):
    fig, ax = plt.subplots()
    
    gap = 0.2 * (b-a)
    
    fig.set_figheight(10)
    fig.set_figwidth(15)
    
    x = np.linspace(a-gap, b+gap)
    y = f(x)
    ax.plot(x, y, 'r', linewidth=1)  
    
    fig.text(0.9, 0.1, '$x$')
    fig.text(0.12, 0.9, '$y$')

    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    ax.xaxis.set_ticks_position('bottom')

    ax.set_xticks((a, b))
    ax.set_xticklabels(('$a$', '$b$'))
    ax.set_yticks([])

    h = (b-a) / k
    
    for i in range(0, k):
        x0 = a + i*h
        x1 = x0 + h
        
         # Cuadratura
        xq = np.linspace(x0, x1)
        iq = q(xq,x0,x1,n)
        ax.plot(xq, iq, 'k', linewidth=1)

        # Función
        iy = f(xq)
        ax.fill_between(xq, 0, iq, facecolor='#0099cc', edgecolor ='k')
        ax.fill_between(xq, iq, iy, hatch = '//', alpha='0.1')    
    plt.show()

In [448]:
ti = {
    0 : np.polynomial.legendre.leggauss(1)[0],
    1 : np.polynomial.legendre.leggauss(2)[0],
    2 : np.polynomial.legendre.leggauss(3)[0],
    3 : np.polynomial.legendre.leggauss(4)[0],  
    4 : np.polynomial.legendre.leggauss(5)[0],
    5 : np.polynomial.legendre.leggauss(6)[0],
    6 : np.polynomial.legendre.leggauss(7)[0]   
}

ci = {
    0 : np.polynomial.legendre.leggauss(1)[1],
    1 : np.polynomial.legendre.leggauss(2)[1],
    2 : np.polynomial.legendre.leggauss(3)[1],
    3 : np.polynomial.legendre.leggauss(4)[1],  
    4 : np.polynomial.legendre.leggauss(5)[1],
    5 : np.polynomial.legendre.leggauss(6)[1],
    6 : np.polynomial.legendre.leggauss(7)[1] 
}

cNCC = {
    0: [1], #rectángulo apoyado en a
    1: [1/2, 1/2], #trapecio
    2: [1/3, 4/3, 1/3], #simpson
    3: [3/8, 9/8, 9/8, 3/8], #simpson 3/8
    4: [14/45, 64/45, 24/45, 64/45, 14/45], #boole
    5: [5*19/288, 5*75/288, 5*50/288, 5*50/288, 5*75/288, 5*19/288],
    6: [41/140, 216/140, 27/140, 272/140, 27/140, 216/140, 41/140]
}

cNCA = {
    0: [1], #punto medio
    1: [3/2, 3/2], #trapecio abierto
    2: [8/3, -4/3, 8/3], 
    3: [55/24, 5/24, 5/24, 55/24], 
    4: [6*11/20, -6*14/20, 6*26/20, -6*14/20, 6*11/20],
    5: [7*611/1440, -7*453/1440, 7*562/1440, 7*562/1440, -7*453/1440, 7*611/1440],
    6: [8*460/945, -8*954/945, 8*2196/945, -8*2459/945, 8*2196/945, -8*954/945, 8*460/945]
}

In [449]:
def NCC(n,a,b,k):
    h = (b-a) / k
    Q = 0
    if (n==0):
        for j in range (0, k):  
            xj = a + j * h
            Q += cNCC[n][0]*f(xj)
        return Q*h            
    else:        
        for j in range (0, k):  
            q = 0
            xj = a + j * h
            if (xj==a): 
                q += cNCC[n][0]*f(xj)
            else:
                q += 2*cNCC[n][0]*f(xj)
            for i in range (1,n):
                xi = xj + i * h / n
                q += cNCC[n][i]*f(xi)
            if (xj + h == b):
                q += cNCC[n][n]*f(xj+h)
            Q += q
        return Q*h/n

In [450]:
def NCA(n,a,b,k):
    h = (b-a) / k
    Q = 0
    if (n==0):
        for j in range (0, k):  
            xj = a + j * h
            Q += cNCA[n][0]*f(xj+h/2)
        return Q*h            
    else:        
        for j in range (0, k):  
            xj = a + j * h
            for i in range (1,n+2):
                xi = xj + i * h / (n+2)              
                Q += cNCA[n][i-1]*f(xi)
        return Q*h/(n+2)
    
def GL(n,a0,b0,k):
    h = (b0-a0)/k
    Q = 0
    for j in range (0,k):
        a = a0 + h*j
        b = a + h
        q = 0
        for i in range (len(ti[n])):
            xi = (b+a)/2 + h*ti[n][i]/2
            q += ci[n][i] * f(xi)
        Q += h*q/2
    return (Q)

In [451]:
def lagrange(x,X,Y):
    n = len(X)
    sum = 0         
    for i in range (0,n):
        prod = Y[i]
        for j in range (0,n):
            if (i!=j): prod *= (x-X[j])/(X[i]-X[j])
        sum += prod
    return sum

def qgl (x, a, b, n):
    if (n==0):
        return f((a+b)/2)*(x**0)
    else:
        X = np.zeros(n+1)
        Y = np.zeros(n+1)    
        for i in range (len(ti[n])):
            X[i] = (b+a)/2 + (b-a)*ti[n][i]/2   
            Y[i] = f(X[i])
        return lagrange(x, X, Y)  

In [452]:
def qncc(x,a,b,n):
    if (n==0):
        return f(a)*(x**0)
    else:
        h = (b-a) / n
        X = np.zeros(n+1)
        Y = np.zeros(n+1)
    for i in range (0,n+1):
        X[i] = a + i * h
        Y[i] = f(X[i])  
    return lagrange(x, X, Y)
 
def qnca(x,a,b,n):
    if (n==0):
        return f((a+b)/2)*(x**0)
    else:
        h = (b-a) / (n+2)
        X = np.zeros(n+1)
        Y = np.zeros(n+1)
    for i in range (1,n+2):
        X[i-1] = a + i * h
        Y[i-1] = f(X[i-1])
    return lagrange(x, X, Y)   

In [459]:
kSlider = widgets.IntSlider(value=1, min=1, max=8, step=1, description=('Intervalos'));
nSlider = widgets.IntSlider(value=1, min=1, max=7, step=1, description=('Puntos (n+1)'));
metodoDropdown = widgets.Dropdown(
    options=['Seleccionar Método','Newton-Cotes Cerrado', 'Newton-Cotes Abierto', 'Gauss-Legendre'],
    description='Graficar:',
    disabled=False
)


def GaussInteractivo(n, k, m):
    n = n-1
    if (m=='Gauss-Legendre'):
        graficoActualizar(a,b,f,qgl,k,n)
    elif (m=='Newton-Cotes Abierto'):
        graficoActualizar(a,b,f,qnca,k,n)
    elif (m=='Newton-Cotes Cerrado'):
        graficoActualizar(a,b,f,qncc,k,n)        
    solNCC = NCC(n, a, b, k)
    solNCA = NCA(n, a, b, k)
    solGL = GL(n, a, b, k)
    df = pd.DataFrame({'Integral Exacta': [integralExacta, 0 ],
                       'N-C Cerrado': [solNCC, ErrorRel(solNCC)],
                       'N-C Abierto': [solNCA, ErrorRel(solNCA)],
                       'Gauss-Legendre': [solGL, ErrorRel(solGL)]},
                      index=['Aproximación', 'Error Porcentual'])    
    display(df)

In [460]:
widget=interactive(GaussInteractivo, n=nSlider, k=kSlider, m=metodoDropdown)
controls = HBox([metodoDropdown, nSlider, kSlider], layout = Layout(flex_flow='row wrap'))
output = widget.children[-1]
display(VBox([controls, output]))